In [83]:
import os

import geopandas as gpd

import json
from shapely.geometry import shape, Point

import pandas as pd
from math import cos, asin, sqrt
import matplotlib.pyplot as plt
import random
import math
import sys
import copy

import math 



In [84]:
# loads the info retrieved from UBER movement API

# sourceid : represents the origin zat
# dstid : represents the destination zat
# hod : hour of the day
# mean_travel_time : mean travel time in seconds
# standard_deviation_travel_time : standard deviation of travel time in seconds
# geometric_mean_travel_time : geometric mean travel time in seconds
# geometric_standard_deviation_travel_time : geometric standard deviation of travel time in seconds

datos=pd.DataFrame(pd.read_csv("../data/preprocess/inputs/bogota-cadastral-2019-3-OnlyWeekdays-HourlyAggregate.csv"))
datos.head()
len(datos)

7060948

# loads the maps of the city

In [91]:
data=gpd.read_file('../data/preprocess/inputs/bogota_geo.json')
    
with open('../data/preprocess/inputs/limites_geojson.json') as f:
	data_lims = json.load(f)   

# EXCLUDES THE SCATTERED AND RURAL ZONES
data=data[data.scatipo.eq(0)]

# computes a new column with the representative point of each zone
data['representative_point']=data['geometry'].apply(lambda x: shape(x).representative_point().xy)

data.dropna(subset=['representative_point'],inplace=True)


# creates a single dataframe with the representative points of the zones and the uber data

In [86]:
datos['source_coordinates']=datos['sourceid'].map(data.set_index('cartodb_id').to_dict()['representative_point'])
datos['dst_coordinates']=datos['dstid'].map(data.set_index('cartodb_id').to_dict()['representative_point'])

datos.dropna(subset=['source_coordinates','dst_coordinates'],inplace=True)

datos['source_coordinates']=datos['source_coordinates'].apply(lambda x: [x[0][0],x[1][0]])
datos['dst_coordinates']=datos['dst_coordinates'].apply(lambda x: [x[0][0],x[1][0]])
datos.head()

,sourceid,dstid,hod,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time,source_coordinates,dst_coordinates
0,360,563,10,1282.33,315.56,1246.33,1.27,"[-74.09962323366398, 4.6862625]","[-74.08758793333334, 4.7211485]"
3,75,548,13,2043.50,377.27,2010.83,1.19,"[-74.03343874582688, 4.6849430000000005]","[-74.21490355950878, 4.625836]"
4,1152,1131,0,514.12,383.08,428.93,1.74,"[-74.11164411332437, 4.677749]","[-74.18004646566523, 4.589392]"
5,1151,1141,0,399.07,208.18,361.21,1.53,"[-74.09159054158111, 4.707669]","[-74.21120185102613, 4.6119699999999995]"
6,351,653,10,1763.42,396.84,1721.73,1.24,"[-74.06838855415091, 4.692584]","[-74.0373346847826, 4.6808475000000005]"


# displays the are being considered

In [112]:
data_gpd = gpd.GeoDataFrame(data)
data_json = data_gpd._to_geo()
# save the data to a json file
# convert dict to json



In [107]:
# displays the data of the map in folium
import folium
from folium import plugins

# Create a map centered in Bogota
mapa = folium.Map(location=[4.6097100, -74.0817500], zoom_start=11)

with open('../data/preprocess/inputs/bogota_geo.json') as f:
	data_lims = json.load(f)

# saves data to json file


# place the data of the map in the folium map
folium.GeoJson(data_lims).add_to(mapa)


# Display the map
#mapa



# Using OSRM we are going to compute the distances between the representative points of the zones

# loads the distances between the zones

# Creates the container for the distances using colombia-latest.osrm
```sh 

docker run -t -v "${PWD}:/data" osrm/osrm-backend osrm-extract -p /opt/car.lua /data/colombia-latest.osm.pbf
```

### Partitions the container
```sh 
docker run -t -v "${PWD}:/data" osrm/osrm-backend osrm-partition /data/colombia-latest.osrm
docker run -t -v "${PWD}:/data" osrm/osrm-backend osrm-customize /data/colombia-latest.osrm
```

# serve in port 5001

```sh 
docker run -t -i -p 5001:5000 -v "${PWD}:/data" osrm/osrm-backend osrm-routed --algorithm mld --max-table-size 100000000 /data/colombia-latest.osrm
```


In [88]:
# custom osrm client that receives multiple origins and destinations
#!curl "http://127.0.0.1:5001/table/v1/driving/53.27528,-2.1271;57.06322,-2.13991;57.1913,-2.12272"

import requests

class OSRMClient:
    def __init__(self, host):
        self.host = host


    
    def get_distance_matrix(self, coordinates,source):
        steps=min(100,math.ceil(len(coordinates)/10))
        distances= {}
    
        
        # ?annotations=distance,duratio
        url = f"http://{self.host}/table/v1/driving/{';'.join([f'{c[0]},{c[1]}' for c in coordinates])}?sources={source}&annotations=distance"
        #print(url)
        response = requests.get(url)
        return response.json()
        


    def get_nearest(self, coordinates):
        url = f"http://{self.host}/nearest/v1/driving/{coordinates[0]},{coordinates[1]}"
        response = requests.get(url)
        coordinates = response.json()["waypoints"][0]["location"]
        return coordinates
    
    def get_distance(self, a, b):
        url = f"http://{self.host}/route/v1/driving/{a[0]},{a[1]};{b[0]},{b[1]}"
        response = requests.get(url)
        return response.json()["routes"][0]["distance"]
    

# computes the distance between two points using the OSRM client

a=[-75.5184621,5.056982]
client = OSRMClient("127.0.0.1:5001")

response = client.get_nearest(a)
print(response)










[-75.518731, 5.057174]


In [89]:
import osrm

client = osrm.Client(host='http://localhost:5001')

def get_distance(origin,destination):
    # receives the coordinates of the origin and destination
    # returns the distance in km

    response = client.route(
        coordinates=[origin, destination],
        overview=osrm.overview.full)
    return response['routes'][0]['distance']/1000

# computes the distances between the zones

To make this process faster we are going to create a dictionary with the distances between the zones using source and destination as keys

In [90]:
datos['source_dst']=datos.apply(lambda x: (x['sourceid'],x['dstid']),axis=1)

client = OSRMClient("127.0.0.1:5001")

dict_coordinates=datos[['sourceid','source_coordinates']].drop_duplicates(subset=['sourceid']).set_index('sourceid').to_dict()['source_coordinates']

dict_new_coordinates={}
for key in dict_coordinates:
    dict_new_coordinates[key]=client.get_nearest(dict_coordinates[key])

datos["new_source_coordinates"]=datos['sourceid'].map(dict_new_coordinates)
datos["new_dst_coordinates"]=datos['dstid'].map(dict_new_coordinates)
datos.head()    

zat_coordinates=datos[['sourceid','new_source_coordinates']].drop_duplicates(subset=['sourceid']).set_index('sourceid').to_dict()['new_source_coordinates']
zat_coordinates.update(datos[['dstid','new_dst_coordinates']].drop_duplicates(subset=['dstid']).set_index('dstid').to_dict()['new_dst_coordinates'])
len(datos)

5075459

In [61]:
import math
# drop from the dictionary the zats that are not in the data or have nan values
zat_coordinates = {k: v for k, v in zat_coordinates.items() if isinstance(v, list)}

In [ ]:
sorted_zat_coordinates = dict(sorted(zat_coordinates.items()))
id_to_position = {i: k for i, k in enumerate(sorted_zat_coordinates.keys())}
id_to_position

In [75]:
client = OSRMClient("127.0.0.1:5001")
test=[pos for pos in sorted_zat_coordinates.values()]

distances={}
for i in range(len(sorted_zat_coordinates)):
    dst=client.get_distance_matrix(test,i)
    for j in range(len(test)):
        distances[(id_to_position[i],id_to_position[j])]=dst['distances'][0][j]

In [76]:
dict_distances=distances.copy()


In [77]:
datos['distance']=datos['source_dst'].apply(lambda x: dict_distances.get(x,None))

In [78]:
datos.to_pickle('../data/preprocess/outputs/travel_time_with_distances.pkl')

In [81]:
datos.shape


(5075459, 13)

In [80]:
datos.groupby('sourceid').agg({'dstid':'count'}).sort_values('dstid',ascending=False)

,dstid
sourceid,
60,15740
644,14926
871,14511
819,14455
327,14419
...,...
983,2
385,2
679,2
